In [31]:
import pandas as pd
import numpy as np

#import packages for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Define the color palette
palette = {0: 'indianred', 1: 'steelblue'}

#load the dataset "final_dataset_Nov_9.csv"
df = pd.read_csv("../Lee/lee_full_dataset.csv")

df.drop(columns=['MAJOR'],inplace=True)
df.rename(columns={'GRADUATE':'Y'},inplace=True)
df = df.astype(int)
df

,STUDENT,104,105,140,143,145,150,151,160,165,...,317,350,365,373,385,414,415,435,436,Y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,5,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9369,13059,0,0,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9370,13060,0,0,-1,0,0,-3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9371,13061,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9372,13062,0,0,0,0,0,-4,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
courses = df.columns.to_list()[1:-1]
print(len(courses))

28


In [33]:
grouped = df.groupby(courses)
df_grouped = pd.DataFrame({
    'Pr(X)' : grouped.size() / len(df),
    'Pr(Y|X)': grouped['Y'].mean()}).reset_index()
df_grouped

,104,105,140,143,145,150,151,160,165,166,...,350,365,373,385,414,415,435,436,Pr(X),Pr(Y|X)
0,-8,0,-5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,1.0
1,-8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000320,1.0
2,-8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,0.0
3,-8,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,0.0
4,-7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.001067,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2361,8,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,0.0
2362,8,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,1.0
2363,8,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000213,1.0
2364,8,0,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000213,1.0


In [34]:
#compare the dot product of Pr(Y|X) and Pr(X) with the mean value of Y in df
np.dot(df_grouped['Pr(Y|X)'], df_grouped['Pr(X)']), df['Y'].mean()

(np.float64(0.5138681459355668), np.float64(0.5138681459355665))

In [37]:
#add a column named "ERROR(Y|X)" which contains the minimum of Pr(Y|X) and 1-Pr(Y|X)
df_grouped['ERROR(Y|X)'] = np.minimum(df_grouped['Pr(Y|X)'], 1-df_grouped['Pr(Y|X)'])

In [38]:
#compute the bayes error rate. This is the expected value of ERROR(Y|X) over the distribution of X
bayes_error_rate = np.dot(df_grouped['ERROR(Y|X)'], df_grouped['Pr(X)'])

In [44]:
print(f'Maximum achievable accuracy: {(1-bayes_error_rate)*100:.3f}%')

Maximum achievable accuracy: 73.437%


In [45]:
65/73

0.8904109589041096

In [25]:
df_grouped.sum()

104         358.000000
105         105.000000
140         607.000000
143         562.000000
145          76.000000
150         344.000000
151         124.000000
160         190.000000
165        1319.000000
166        1696.000000
201        1978.000000
207        1363.000000
265        1989.000000
266         964.000000
267        1510.000000
301        1625.000000
302         358.000000
304         589.000000
314         770.000000
317        2094.000000
350         540.000000
365         488.000000
373         582.000000
385         864.000000
414        1706.000000
415         430.000000
435         890.000000
436         966.000000
COUNT      9374.000000
Pr(Y|X)    1278.793426
dtype: float64

In [24]:
df_grouped['COUNT'].value_counts().sort_index()

COUNT
1      1838
2       165
3        95
4        40
5        31
       ... 
198       1
207       1
212       1
363       1
372       1
Name: count, Length: 68, dtype: int64

In [ ]:
#create a new dataframe df_grouped as fo

In [ ]:
temp_df = df.drop(columns=['STUDENT'])

df_grouped = df['Y']
#add a column to df_grouped named 'X'. Each entry should be the row in df[courses] corresponding to that row in df_grouped
df_grouped['X'] = df[courses].values.tolist()

#group df_grouped by 'X' and calculate the mean of 'Y' for each group. Store the result in a new column 'Pr(Y|X)' in df_grouped

temp_df['Pr(Y|X)'] = 0.0

temp_df['Pr(Y|X)'] = temp_df.groupby(by=courses)['Y'].mean()

In [13]:
df_unstacked = df.melt(id_vars='STUDENT', 
                       value_vars=courses, 
                       var_name='CRSE', 
                       value_name='SEM_SIGNED')

In [16]:
df_unstacked.sort_values(by=['STUDENT', 'CRSE'], inplace=True)
df_unstacked

,STUDENT,CRSE,SEM_SIGNED
0,0,104,0
9374,0,105,0
18748,0,140,0
28122,0,143,0
37496,0,145,0
...,...,...,...
224975,13063,385,0
234349,13063,414,0
243723,13063,415,0
253097,13063,435,0


In [17]:
#check how many rows are there correpsonding to each student
df_unstacked.groupby('STUDENT').size().value_counts()

28    9374
Name: count, dtype: int64